Live data
=========

For optimum results this notebook should be run during the Forex trading session.

In [1]:
%gui qt5

from ib_insync import *
util.useQt()

ib = IB()
ib.connect('127.0.0.1', 7497, clientId=15)

Create some Forex contracts:

In [2]:
contracts = [Forex(pair) for pair in 'EURUSD USDJPY GBPUSD USDCHF USDCAD AUDUSD'.split()]

eurusd = contracts[0]

Request streaming ticks for them:

In [3]:
for contract in contracts:
    ib.reqMktData(contract, '', False, False)

Wait a few seconds for the tickers to get filled.

In [5]:
ticker = ib.ticker(eurusd)
ib.sleep(2)

ticker

Ticker(contract=Forex('EURUSD', exchange='IDEALPRO'), time=datetime.datetime(2017, 7, 17, 17, 59, 44, 739157, tzinfo=datetime.timezone.utc), bid=1.14775, bidSize=19674000, ask=1.1478, askSize=1000000, lastSize=0, prevBid=1.1478, prevBidSize=18674000, prevAsk=1.14785, prevAskSize=4000000, high=1.14875, low=1.1435, close=1.1469, domBids=[], domAsks=[])

In [6]:
ticker.ticks[0]

TickData(time=datetime.datetime(2017, 7, 17, 17, 59, 18, 487137, tzinfo=datetime.timezone.utc), tickType=0, price=nan, size=500000)

The price of Forex ticks is always nan. To get a midpoint price use ``marketPrice()``.

The tickers are kept
live updated, try this a few times to see if the price changes:

In [7]:
ticker.marketPrice()

1.147775

The following cell will start an endless loop that prints a live updated ticker table.
It is updated with every new network packet that arrivess.

The loop can be interupted with "interupt kernel" in Jupyter. The notebook will keep running and the loop can be
started again.

In [ ]:
from IPython.display import display, clear_output
import pandas as pd

df = pd.DataFrame(columns='symbol bidSize bid ask askSize high low close'.split())
df['symbol'] = [c.symbol + c.currency for c in contracts]

while ib.waitOnUpdate():
    for i, t in enumerate(ib.tickers()):
        df.iloc[i, 1:] = (
                t.bidSize, t.bid, t.ask, t.askSize, t.high, t.low, t.close)
    clear_output(wait=True)
    display(df)

,symbol,bidSize,bid,ask,askSize,high,low,close
0,EURUSD,14674000,1.1478,1.14785,5000000,1.14875,1.1435,1.1469
1,USDJPY,26000000,112.63,112.64,11000000,112.87,112.33,112.54
2,GBPUSD,1152000,1.3059,1.30595,1000000,1.3113,1.3051,1.31
3,USDCHF,4200000,0.9621,0.96215,1000000,0.9659,0.95945,0.96365
4,USDCAD,7200000,1.2687,1.26875,1000000,1.26975,1.26275,1.2647
5,AUDUSD,16755000,0.7803,0.7804,8000000,0.78385,0.7802,0.78285


To stop the subscription:

In [13]:
for contract in contracts:
    ib.cancelMktData(contract)